In [65]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import time
from bs4 import BeautifulSoup
import re
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, NoSuchFrameException

In [104]:
import pandas as pd
import numpy as np

df = pd.DataFrame(columns=['cafe_name', "cafe_type", 'starring', 'visitor_review', 'blog_review', 'address', 'business_hours', 'phone_number', 'connection_site', 'description',
"menu", "keyword", 'keyword_number'])

In [58]:
cafe_name_list = ['우주라이크커피 경산점',
 '하늘아래사진관',
 '말그미옥상',
 '호미화방',
 '코너커피',
 '줄라이모닝',
 '카페로커피',
 '엠프티커피',
 '다올',
 '텀블루']

In [70]:
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException

def get_new_reference(driver, xpath):
    """
    페이지나 프레임 전환 후 요소를 다시 찾아서 반환합니다.
    """
    try:
        return WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
        print("Timeout while waiting for element.")
        return None

In [59]:
def cafe_crawling(driver):
#     try:
#         WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "place_section no_margin OP4V8")))
#         time.sleep(1)
#     except:
#         driver = switch_driver(driver, "searchIframe")
#         print(1)
#         return None
    time.sleep(1.5)
    try:
        description_a_link = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[5]/div/div[2]/div/div/div[7]/div/a/span[1]')
        description_a_link.click()
    except Exception as ex:
        pass

    # 기본 카페 정보 크롤링
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    address = soup.find('span', class_='LDgIH').text if soup.find('span', class_='LDgIH') else ""
    if not "대구" in address and not "경산" in address:
        return None

    cafe_name = soup.find('span', class_='Fc1rA').text
    cafe_type = soup.find('span', class_='DJJvD').text
    reviews_div = soup.find('div', class_='dAsGb')
    starring = float(reviews_div.find('span', class_='PXMot LXIwF').text.strip()[2:]) if reviews_div.find('span', class_='PXMot LXIwF') else 0
    
    if not reviews_div.find_all('span', class_='PXMot') or len(reviews_div.find_all('span', class_='PXMot')) < 3:
        return None
    
    visitor_reviews = int(reviews_div.find_all('span', class_='PXMot')[1 if starring else 0].text.strip().split(" ")[1].replace(",", ""))
    blog_reviews = int(reviews_div.find_all('span', class_='PXMot')[2 if starring else 1].text.strip().split(" ")[1].replace(",", ""))
    
    if visitor_reviews < 10:
        return None
    
    description = soup.find_all('span', class_='zPfVt')[1].text if len(soup.find_all('span', class_='zPfVt')) > 1 else "" 
    address = soup.find('span', class_='LDgIH').text if soup.find('span', class_='LDgIH') else ""
    business_hours = soup.find('time').text if soup.find('time') else "" 
    phone_number = soup.find('span', class_='xlx7Q').text if soup.find('span', class_='xlx7Q') else ""

    # 키워드로 넘어가기
    keyword_tab_link = driver.find_element(By.XPATH, "//div[@class='flicking-camera']/a[.//span[contains(text(), '리뷰')]]")
    keyword_tab_link.click()
    keyword_list_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Tvx37")))
    keyword_list_link.click()

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    keyword_number = int(soup.find('em').text)

    keywords = {}
    ui_tags = soup.find('ui', class_='uNsI9')
    li_tags = soup.find_all('li', class_='nbD78')

    for li in li_tags:
        # 첫 번째 숫자를 int 형으로 변환
        text = li.find('span', class_='nWiXa').text.strip('"')
        numbers = re.findall(r'\d+', li.find('span', class_='TwM9q').text)
        number = int(numbers[0]) if numbers else None
        keywords[text] = number
    
    return [cafe_name, cafe_type, starring, visitor_reviews, blog_reviews, address, business_hours, phone_number, "", description, "", keywords, keyword_number]

In [60]:
def switch_driver(driver, iframe):
    try:
        driver.switch_to.default_content()
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, iframe)))
    except NoSuchFrameException:
        print(f"Frame '{iframe}' not found.")
    return driver

In [61]:
# def switch_iframe(driver):
#     try:
#         # 현재 iframe 확인
#         current_iframe = driver.find_element(By.TAG_NAME, 'iframe').get_attribute('id')

#         # 현재 iframe이 'searchIframe'인 경우 'entryIframe'으로 전환
#         if current_iframe == 'searchIframe':
#             driver.switch_to.default_content() # 메인 컨텐츠로 돌아감
#             driver.switch_to.frame('entryIframe') # 'entryIframe'으로 전환

#         # 현재 iframe이 'entryIframe'인 경우 'searchIframe'으로 전환
#         elif current_iframe == 'entryIframe':
#             driver.switch_to.default_content() # 메인 컨텐츠로 돌아감
#             driver.switch_to.frame('searchIframe') # 'searchIframe'으로 전환

#     except NoSuchElementException:
#         # iframe을 찾을 수 없는 경우, 'searchIframe'으로 전환 시도
#         try:
#             driver.switch_to.default_content() # 메인 컨텐츠로 돌아감
#             driver.switch_to.frame('searchIframe') # 'searchIframe'으로 전환
#         except NoSuchElementException:
#             # 'searchIframe'도 찾을 수 없는 경우
#             print("searchIframe을 찾을 수 없습니다.")

In [71]:
# Selenium을 사용하여 웹 드라이버 초기화
driver = webdriver.Chrome()

#... 이전 코드 ...

for cafe_name in cafe_name_list:
    # 검색
    driver.get("https://map.naver.com/p/search/" + cafe_name)
    time.sleep(1.5)  # 추가 대기 시간
    
    current_url = driver.current_url
    url_point = current_url.split("/")

    if len(url_point) <= 6:
        driver = switch_driver(driver, "searchIframe")
        scrollable_div = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "_pcmap_list_scroll_container")))

        for _ in range(5):
            driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
            time.sleep(1)  # 적절한 로딩 시간 기다림

        li_elements = scrollable_div.find_elements(By.TAG_NAME, "li") 

        for idx in range(len(li_elements)):
            try:
                # 페이지나 프레임 전환 후 요소를 다시 찾습니다.
                scrollable_div = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "_pcmap_list_scroll_container")))
                cafe_link = scrollable_div.find_element(By.XPATH, f'./ul/li[{idx+1}]/div[1]/div[2]/a[1]')
                cafe_link.click()

                driver = switch_driver(driver, "entryIframe")

                cafe_row = cafe_crawling(driver)
                if cafe_row:
                    df.loc[len(df)] = cafe_row
                    print(cafe_row)

                driver = switch_driver(driver, "searchIframe")
            except StaleElementReferenceException:
                # 요소가 더 이상 유효하지 않을 경우 다시 시도
                continue
    else:   
        driver = switch_driver(driver, "entryIframe")
        cafe_row = cafe_crawling(driver)
        if cafe_row:
            df.loc[len(df)] = cafe_row
            print(cafe_row)


['우주라이크커피 경산점', '카페', 4.77, 84, 4, '경북 경산시 대학로10길 24 1층', '21:00에 영업 종료', '053-710-8852', '', '반갑습니다 : ) 우주라이크 커피 경산점 인사드립\n니다! 새롭고, 신선하고, 합리적인 커피! 브라질 내\n추럴, 콜롬비아 수프리모 워시드, 에티오피...', '', {'커피가 맛있어요': 34, '친절해요': 23, '디저트가 맛있어요': 18, '음료가 맛있어요': 15, '가성비가 좋아요': 12, '특별한 메뉴가 있어요': 9, '매장이 청결해요': 9, '대화하기 좋아요': 7, '주차하기 편해요': 6, '좌석이 편해요': 5}, 61]
['말그미옥상', '케이크전문', 4.55, 91, 153, '대구 남구 계명중앙1길 25 2층', '20:00에 영업 종료', '0507-1310-3966', '', "'맑음' 이라는 말에서 따온 순 우리말\n말그미라는 단어를 사용한 '말그미옥상' 입니다.", '', {'커피가 맛있어요': 35, '디저트가 맛있어요': 17, '대화하기 좋아요': 17, '음료가 맛있어요': 16, '친절해요': 15, '인테리어가 멋져요': 10, '매장이 청결해요': 10, '뷰가 좋아요': 10, '특별한 메뉴가 있어요': 6, '사진이 잘 나와요': 4}, 69]
Stale element reference. Attempting to find element again.
Timeout while waiting for element.
Stale element reference. Attempting to find element again.
Timeout while waiting for element.
Timeout while waiting for element.
Timeout while waiting for element.
Timeout while waiting for element.
Timeout while waiting for element.
Timeout w

In [68]:
df.tail(10)

,cafe_name,cafe_type,starring,visitor_review,blog_review,address,business_hours,phone_number,connection_site,description,menu,keyword,keyword_number
0,우주라이크커피 경산점,카페,4.77,84,4,경북 경산시 대학로10길 24 1층,21:00에 영업 종료,053-710-8852,,"반갑습니다 : ) 우주라이크 커피 경산점 인사드립\n니다! 새롭고, 신선하고, 합리...",,"{'커피가 맛있어요': 34, '친절해요': 23, '디저트가 맛있어요': 18, ...",61
1,말그미옥상,케이크전문,4.55,91,153,대구 남구 계명중앙1길 25 2층,20:00에 영업 종료,0507-1310-3966,,'맑음' 이라는 말에서 따온 순 우리말\n말그미라는 단어를 사용한 '말그미옥상' 입니다.,,"{'커피가 맛있어요': 35, '디저트가 맛있어요': 17, '대화하기 좋아요': ...",69
2,우주라이크커피 경산점,카페,4.77,84,4,경북 경산시 대학로10길 24 1층,21:00에 영업 종료,053-710-8852,,"반갑습니다 : ) 우주라이크 커피 경산점 인사드립\n니다! 새롭고, 신선하고, 합리...",,"{'커피가 맛있어요': 34, '친절해요': 23, '디저트가 맛있어요': 18, ...",61
3,말그미옥상,케이크전문,4.55,91,153,대구 남구 계명중앙1길 25 2층,20:00에 영업 종료,0507-1310-3966,,'맑음' 이라는 말에서 따온 순 우리말\n말그미라는 단어를 사용한 '말그미옥상' 입니다.,,"{'커피가 맛있어요': 35, '디저트가 맛있어요': 17, '대화하기 좋아요': ...",69
4,우주라이크커피 경산점,카페,4.77,84,4,경북 경산시 대학로10길 24 1층,21:00에 영업 종료,053-710-8852,,"반갑습니다 : ) 우주라이크 커피 경산점 인사드립\n니다! 새롭고, 신선하고, 합리...",,"{'커피가 맛있어요': 34, '친절해요': 23, '디저트가 맛있어요': 18, ...",61
5,말그미옥상,케이크전문,4.55,91,153,대구 남구 계명중앙1길 25 2층,20:00에 영업 종료,0507-1310-3966,,'맑음' 이라는 말에서 따온 순 우리말\n말그미라는 단어를 사용한 '말그미옥상' 입니다.,,"{'커피가 맛있어요': 35, '디저트가 맛있어요': 17, '대화하기 좋아요': ...",69
6,엠프티커피,"카페,디저트",4.59,329,233,대구 남구 대경길 185,22:00에 영업 종료,0507-1359-6607,,,,"{'커피가 맛있어요': 220, '디저트가 맛있어요': 177, '친절해요': 13...",265


In [121]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# WebDriver 초기화
driver = webdriver.Chrome() # Chrome 드라이버 사용
driver.get("https://map.naver.com/p/search/카페로커피") # 네이버 지도 접속

# 결과 로드 대기
time.sleep(3) # 페이지 로드를 위해 잠시 대기

# searchIframe으로 컨텍스트 전환 + div scroll 내리기
driver.switch_to.frame("searchIframe")
scroll_div = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "_pcmap_list_scroll_container")))
for i in range(5):
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scroll_div)


# 목록을 담고 있는 ul 태그를 찾고, 그 안의 모든 li 태그를 가져옴
cafe_list = scroll_div.find_elements(By.TAG_NAME, "li")
cafe_size = len(cafe_list)
idx = 0

while idx < cafe_size:
    # 각 li 태그를 순회하면서 카페이름에 해당하는 a 링크를 클릭
    cafe_list = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "_pcmap_list_scroll_container"))).find_elements(By.TAG_NAME, "li")
    for item in cafe_list[idx:]:
        try:
            cafe_link = item.find_element(By.TAG_NAME, "a")
            cafe_link.click()
            time.sleep(3)

            # 필요한 작업 수행 (예: 상세 정보 수집)
            driver.switch_to.default_content()
            driver.switch_to.frame("entryIframe")
            print(cafe_crawling2())
    #         cafe_row = cafe_crawling(driver)
    #         if cafe_row:
    #             df.loc[len(df)] = cafe_row
    #             print(cafe_row)

            # 다음 카페로 이동하기 전에 searchIframe으로 다시 컨텍스트 전환
            driver.switch_to.default_content()
            driver.switch_to.frame("searchIframe")
            
            idx += 1
        except Exception as e:
            break

# 원래 프레임으로 돌아감
driver.switch_to.default_content()


None
['마고플레인 평산', '카페,디저트', 4.48, 931, 574, '경북 경산시 삼성현로 587', '21:50에 라스트오더', '0507-1360-7827', '', '카페 평산은 동양적인 컨셉의 편안한 쉼을 제공합니다\n\n지친 일상의 회복이 되도록 노력하는 공간이 되겠습니다 :)', '', {'커피가 맛있어요': 266, '디저트가 맛있어요': 199, '주차하기 편해요': 175, '인테리어가 멋져요': 152, '음료가 맛있어요': 141, '매장이 청결해요': 97, '뷰가 좋아요': 83, '대화하기 좋아요': 80, '친절해요': 72, '사진이 잘 나와요': 70}, 582]
['후버', '카페,디저트', 4.6, 813, 466, '경북 경산시 솔숲길4길 6-14 1층', '22:00에 라스트오더', '0507-1413-0712', '', '안녕하세요 당신의 오아시스, 후버입니다\n\nhoover는 이곳을 좋아해 주실 모든 분께 열려있습니다\n황량한 사막에도 어딘가엔 목을 축이고 쉬어갈 수 있는 오아시스가 존재합니다 사막의 오아시스처럼 쉼이 생각날 때면 언제나 찾아올 수 있는 쉼터 같은 공간이 되었으면 합니다\n\nhoover는 지구를 지키기 위해 모든 일회용품은 친환경제품으로 사용 중이며 직원들 모두가 환경보호를 위해 노력 중입니다\n지구와 함께 쉬어가는 후버로 기억될 수 있도록 변함없이 꾸준히 지켜나가겠습니다\n\nwhoever likes here!\nwhatever saves earth!\nhere is hoover.', '', {'커피가 맛있어요': 239, '디저트가 맛있어요': 132, '인테리어가 멋져요': 124, '친절해요': 100, '매장이 청결해요': 98, '음료가 맛있어요': 91, '대화하기 좋아요': 91, '특별한 메뉴가 있어요': 49, '좌석이 편해요': 45, '집중하기 좋아요': 43}, 458]
['아틀리에빈계양점', '카페', 4.6, 682, 15, '경북 경산시 백양로 100', '21

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6FBBC82B2+55298]
	(No symbol) [0x00007FF6FBB35E02]
	(No symbol) [0x00007FF6FB9F05AB]
	(No symbol) [0x00007FF6FBA3175C]
	(No symbol) [0x00007FF6FBA318DC]
	(No symbol) [0x00007FF6FBA6CBC7]
	(No symbol) [0x00007FF6FBA520EF]
	(No symbol) [0x00007FF6FBA6AAA4]
	(No symbol) [0x00007FF6FBA51E83]
	(No symbol) [0x00007FF6FBA2670A]
	(No symbol) [0x00007FF6FBA27964]
	GetHandleVerifier [0x00007FF6FBF40AAB+3694587]
	GetHandleVerifier [0x00007FF6FBF9728E+4048862]
	GetHandleVerifier [0x00007FF6FBF8F173+4015811]
	GetHandleVerifier [0x00007FF6FBC647D6+695590]
	(No symbol) [0x00007FF6FBB40CE8]
	(No symbol) [0x00007FF6FBB3CF34]
	(No symbol) [0x00007FF6FBB3D062]
	(No symbol) [0x00007FF6FBB2D3A3]
	BaseThreadInitThunk [0x00007FFA4970257D+29]
	RtlUserThreadStart [0x00007FFA4B3EAA78+40]


In [113]:
def cafe_crawling2():
    try:
        description_a_link = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[5]/div/div[2]/div/div/div[7]/div/a/span[1]')
        description_a_link.click()
    except Exception as ex:
        pass

    # 기본 카페 정보 크롤링
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    address = soup.find('span', class_='LDgIH').text if soup.find('span', class_='LDgIH') else ""
    if not "대구" in address and not "경산" in address:
        return None

    cafe_name = soup.find('span', class_='Fc1rA').text
    cafe_type = soup.find('span', class_='DJJvD').text
    reviews_div = soup.find('div', class_='dAsGb')
    starring = float(reviews_div.find('span', class_='PXMot LXIwF').text.strip()[2:]) if reviews_div.find('span', class_='PXMot LXIwF') else 0
    
    if not reviews_div.find_all('span', class_='PXMot') or len(reviews_div.find_all('span', class_='PXMot')) < 3:
        return None
    
    visitor_reviews = int(reviews_div.find_all('span', class_='PXMot')[1 if starring else 0].text.strip().split(" ")[1].replace(",", ""))
    blog_reviews = int(reviews_div.find_all('span', class_='PXMot')[2 if starring else 1].text.strip().split(" ")[1].replace(",", ""))
    
    if visitor_reviews < 10:
        return None
    
    description = soup.find_all('span', class_='zPfVt')[1].text if len(soup.find_all('span', class_='zPfVt')) > 1 else "" 
    address = soup.find('span', class_='LDgIH').text if soup.find('span', class_='LDgIH') else ""
    business_hours = soup.find('time').text if soup.find('time') else "" 
    phone_number = soup.find('span', class_='xlx7Q').text if soup.find('span', class_='xlx7Q') else ""

    # 키워드로 넘어가기
    keyword_tab_link = driver.find_element(By.XPATH, "//div[@class='flicking-camera']/a[.//span[contains(text(), '리뷰')]]")
    keyword_tab_link.click()
    keyword_list_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Tvx37")))
    keyword_list_link.click()

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    keyword_number = int(soup.find('em').text)

    keywords = {}
    ui_tags = soup.find('ui', class_='uNsI9')
    li_tags = soup.find_all('li', class_='nbD78')

    for li in li_tags:
        # 첫 번째 숫자를 int 형으로 변환
        text = li.find('span', class_='nWiXa').text.strip('"')
        numbers = re.findall(r'\d+', li.find('span', class_='TwM9q').text)
        number = int(numbers[0]) if numbers else None
        keywords[text] = number
    
    return [cafe_name, cafe_type, starring, visitor_reviews, blog_reviews, address, business_hours, phone_number, "", description, "", keywords, keyword_number]

In [ ]:
# WebDriver 초기화
driver = webdriver.Chrome() # Chrome 드라이버 사용

for cafe_name in cafe_name_list:
    # 검색
    driver.get("https://map.naver.com/p/search/" + cafe_name)
    time.sleep(3) # 페이지 로드를 위해 잠시 대기
    
    current_url = driver.current_url
    url_point = current_url.split("/")

    if len(url_point) <= 6:
        # searchIframe으로 컨텍스트 전환 + div scroll 내리기
        driver.switch_to.frame("searchIframe")
        scroll_div = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "_pcmap_list_scroll_container")))
        scroll_down(driver)

        # 목록을 담고 있는 ul 태그를 찾고, 그 안의 모든 li 태그를 가져옴
        cafe_list = scroll_div.find_elements(By.TAG_NAME, "li") 

        # 각 li 태그를 순회하면서 카페이름에 해당하는 a 링크를 클릭
        for item in cafe_list:
            try:
                cafe_link = item.find_element(By.TAG_NAME, "a")
                cafe_link.click()

                # 필요한 작업 수행 (예: 상세 정보 수집)
                driver.switch_to.default_content()
                driver.switch_to.frame("entryIframe")
                
                cafe_row = cafe_crawling2()
                if cafe_row:
                    df.loc[len(df)] = cafe_row
                    print(cafe_row)

                # 다음 카페로 이동하기 전에 searchIframe으로 다시 컨텍스트 전환
                driver.switch_to.default_content()
                driver.switch_to.frame("searchIframe")
            except Exception as e:
                print(f"An error occurred: {e}")
    else:
        driver.switch_to.frame("entryIframe")
        cafe_row = cafe_crawling2()
        if cafe_row:
            df.loc[len(df)] = cafe_row
            print(cafe_row)
        driver.switch_to.default_content()
        driver.switch_to.frame("searchIframe")

    
